In [1]:
import trino
import pandas as pd
import io
import boto3
from datetime import datetime
from itertools import islice
import numpy as np
from datetime import date, timedelta

In [2]:
# ---- reporting config (edit ONLY this) ----
TABLE_NAME = "power_consumption_report"   # <— change only this

# derived (don’t edit)
REPORT_TABLE = f"adhoc.facts_prod.{TABLE_NAME}"
REPORT_S3_LOCATION = f"s3a://naarni-data-lake/aqua/warehouse/facts_prod.db/{TABLE_NAME}/"

# --------------------
# Step 1: Connect to Trino
# --------------------
print("🔌 [1/5] STEP 1: Connecting to Trino...")

conn = trino.dbapi.connect(
    host="trino",       # just hostname or IP
    port=8080,
    user="admin",
    catalog="adhoc",
    schema="default"
)

print("✅ [1/5] STEP 1: Connected to Trino")


🔌 [1/5] STEP 1: Connecting to Trino...
✅ [1/5] STEP 1: Connected to Trino


In [3]:
# --------------------
# Step 2: Function to fetch data for a given day
# --------------------
def fetch_data_for_day(conn, date_str: str) -> pd.DataFrame:
    print(f"📥 [2/5] STEP 2a: Validating and fetching data for {date_str}...")

    target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
    yesterday_date = target_date - timedelta(days=1)
    
    cursor = conn.cursor()
    print(f"⚙️ [2/5] STEP 2b: Executing query for {target_date} and {yesterday_date}...")
    cursor.execute(f"""
        WITH two_days_data AS (
            SELECT 
                "id",
                at_timezone("timestamp", 'Asia/Kolkata') AS IST,
                "BAT_SOC", "Bat_Voltage", "Total_Battery_Current", "GUN_Connection_Status",
                "OdoMeterReading", "Gear_Position", "Vehiclereadycondition",
                "Chargingcontactor1positive", "Chargingcontactor1negative",
                "Chargingcontactor2positive", "Chargingcontactor2negative"
            FROM
                facts_prod.can_parsed_output_100
            WHERE
                dt = DATE '{target_date.isoformat()}' OR dt = DATE '{yesterday_date.isoformat()}'                
        )
        SELECT
            *
        FROM
            two_days_data
        WHERE
            CAST(IST AS DATE) = DATE '{target_date.isoformat()}'
    """)

    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=columns)
    print(f"📊 [2/5] STEP 2c: Rows fetched for the two-day period: {len(df)}")

    return df

In [4]:
import pandas as pd
import numpy as np

def process_soc_charging_data(df: pd.DataFrame):
    """
    Generates a summary, and performs outlier analysis for charging events
    across multiple device IDs.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing raw data for one or more devices.
        
    Returns:
        pd.DataFrame: A DataFrame summarizing charging events for each device.
    """

    # Create a copy of the DataFrame to avoid modifying a slice.
    df = df.copy()    
    
    if df.empty or 'id' not in df.columns:
        print("Input DataFrame is empty or does not contain an 'id' column.")
        return pd.DataFrame()
        
    device_ids = df['id'].unique().tolist()
    all_summary_data = []

    for device_id in device_ids:
        device_df = df[df['id'] == device_id].copy()

        # Check if the grouped DataFrame is empty
        if device_df.empty:
            print(f"No charging events were detected for device {device_id}.")
            continue        
        
        if 'timestamp' in device_df.columns:
            device_df.loc[:, 'ist_timestamp'] = pd.to_datetime(device_df['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        elif 'IST' in device_df.columns:
            device_df.loc[:, 'ist_timestamp'] = pd.to_datetime(device_df['IST'])
        else:
            print(f"Neither 'timestamp' nor 'IST' column found for device {device_id}. Cannot proceed.")
            continue
            
        device_df.sort_values(by='ist_timestamp', inplace=True)
        device_df.reset_index(drop=True, inplace=True)

        current_threshold = 3200
        device_df = device_df[(device_df['Total_Battery_Current'].abs() != current_threshold)].copy()
        
        for col in ['BAT_SOC', 'Bat_Voltage', 'Chargingcontactor1positive',
                    'Chargingcontactor1negative', 'Chargingcontactor2positive',
                    'Chargingcontactor2negative', 'GUN_Connection_Status']:
            device_df.loc[:, col] = device_df[col].fillna(0)
        
        for col in ['BAT_SOC', 'Bat_Voltage']:
            device_df.loc[:, col] = device_df[col].replace(0.0, np.nan).ffill().bfill()
        
        device_df.loc[:, 'Total_Battery_Current'] = device_df['Total_Battery_Current'].fillna(0)

        # --- MODIFICATION START ---
        # Use GUN_Connection_Status for session detection
        charging_start_indices = device_df[device_df['GUN_Connection_Status'].diff() == 1].index.tolist()
        charging_end_indices = device_df[device_df['GUN_Connection_Status'].diff() == -1].index.tolist()
        # --- MODIFICATION END ---
        
        if not charging_start_indices and not device_df.empty and device_df.iloc[0]['GUN_Connection_Status'] == 1:
            charging_start_indices.insert(0, device_df.index[0])
        
        if len(charging_end_indices) < len(charging_start_indices):
            if len(charging_start_indices) > 0:
                charging_end_indices.append(device_df.index[-1])
            
        merged_events = []
        if len(charging_start_indices) > 0 and len(charging_end_indices) > 0:
            num_sessions = min(len(charging_start_indices), len(charging_end_indices))

            for i in range(num_sessions):
                start_index = charging_start_indices[i]
                end_index = charging_end_indices[i]
                
                if i > 0:
                    prev_end_time = device_df.loc[charging_end_indices[i-1], 'ist_timestamp']
                    current_start_time = device_df.loc[start_index, 'ist_timestamp']
                    prev_end_soc = device_df.loc[charging_end_indices[i-1], 'BAT_SOC']
                    current_start_soc = device_df.loc[start_index, 'BAT_SOC']

                    time_diff = (current_start_time - prev_end_time).total_seconds()
                    soc_diff = abs(current_start_soc - prev_end_soc)
                    
                    if (time_diff <= 5 * 60 and soc_diff <= 1.0) or (time_diff <= 60):
                        merged_events[-1] = (merged_events[-1][0], end_index)
                        continue
                
                merged_events.append((start_index, end_index))
        else:
            print(f"No charging events were detected for device {device_id}.")
            continue

        summary_data_device = []
        BATTERY_CAPACITY_KWH = 423
        
        for start_index, end_index in merged_events:
            event_df = device_df.loc[start_index:end_index].copy()

            if event_df.empty:
                print(f"Warning: Empty event data found for device {device_id}. Skipping.")
                continue            
                
            # MODIFICATION: Use 'GUN_Connection_Status' to filter for charging periods
            charging_periods = event_df[event_df['GUN_Connection_Status'] == 1].copy()
            
            total_duration = 0
            if not charging_periods.empty:
                charging_periods.loc[:, 'time_diff'] = charging_periods['ist_timestamp'].diff().dt.total_seconds().fillna(0)
                total_duration = charging_periods['time_diff'].sum()

            start_row = event_df.iloc[0].copy()
            end_row = event_df.iloc[-1].copy()
            
            energy_Wh = 0
            if not charging_periods.empty:
                charging_periods.loc[:, 'power_W'] = charging_periods['Bat_Voltage'] * charging_periods['Total_Battery_Current'].abs()
                energy_Wh = np.trapezoid(charging_periods['power_W'], x=charging_periods['ist_timestamp'].astype(np.int64) / 10**9) / 3600
            
            total_kwh_consumed_tpc = energy_Wh / 1000

            total_kwh_consumed_soc = (end_row['BAT_SOC'] - start_row['BAT_SOC']) * BATTERY_CAPACITY_KWH / 100
            total_kwh_consumed_soc = abs(total_kwh_consumed_soc)

            percent_diff = 0
            if total_kwh_consumed_tpc + total_kwh_consumed_soc != 0:
                percent_diff = (abs(total_kwh_consumed_tpc - total_kwh_consumed_soc) / 
                                ((total_kwh_consumed_tpc + total_kwh_consumed_soc) / 2)) * 100
            
            summary_data_device.append({
                'vehicle_id': device_id,
                'start_time': start_row['ist_timestamp'],
                'end_time': end_row['ist_timestamp'],
                'charge_dur': round(total_duration,2),
                'soc_start': start_row['BAT_SOC'],
                'soc_end': end_row['BAT_SOC'],
                'tpc_kwh': round(total_kwh_consumed_tpc,2),
                'soc_kwh': round(total_kwh_consumed_soc,2),
                'diff_kw_percent': round(percent_diff,2)
            })
        
        all_summary_data.extend(summary_data_device)
            
    return pd.DataFrame(all_summary_data)

In [5]:
def _quote_ident(ident: str) -> str:
    return '"' + ident.replace('"', '""') + '"'

def _qualify_and_quote(table_fq: str) -> str:
    parts = table_fq.split(".")
    if len(parts) != 3:
        raise ValueError(f"Table must be 'catalog.schema.table', got: {table_fq}")
    return ".".join(_quote_ident(p) for p in parts)

def _trino_type_from_series(s: pd.Series) -> str:
    dtype = str(s.dtype)
    if "datetime64" in dtype:
        return "timestamp(6)"
    if dtype == "bool":
        return "boolean"
    if dtype.startswith("int"):
        return "bigint"
    if dtype.startswith("float"):
        return "double"
    return "varchar"

In [6]:
def write_df_to_iceberg(
    df: pd.DataFrame,
    table: str = REPORT_TABLE,
    s3_location: str = REPORT_S3_LOCATION,
    batch_size: int = 5000,
):
    print("💾 [4/5] STEP 4a: Preparing to write results to Iceberg...")
    cur = conn.cursor()
    fq_table = _qualify_and_quote(table)

    if df is None or df.empty:
        cols = list(df.columns) if df is not None else []
        if cols:
            column_defs = [f'{_quote_ident(col)} {_trino_type_from_series(df[col])}' for col in cols]
        else:
            column_defs = ['"id" varchar']
    else:
        column_defs = [f'{_quote_ident(col)} {_trino_type_from_series(df[col])}' for col in df.columns]

    cols_sql = ",\n        ".join(column_defs)

    create_sql = f"""
    CREATE TABLE IF NOT EXISTS {fq_table} (
        {cols_sql}
    )
    """
    print("🔍 [4/5] STEP 4b: Creating table if not exists...")
    cur.execute(create_sql)
    print(f"🛠️ [4/5] STEP 4c: Ensured Iceberg table {table} exists")

    if df is None or df.empty:
        print("⚠️ [4/5] No rows to insert, skipping insert step")
        cur.close()
        return

    col_idents = ", ".join(_quote_ident(c) for c in df.columns)
    placeholders = ", ".join(["?"] * len(df.columns))
    insert_sql = f"INSERT INTO {fq_table} ({col_idents}) VALUES ({placeholders})"
    print("🔍 [4/5] STEP 4d: Prepared INSERT statement")

    converters = []
    for c in df.columns:
        dtype = str(df[c].dtype)
        if "datetime64" in dtype:
            converters.append(lambda v: (pd.to_datetime(v).tz_localize(None).to_pydatetime()
                                         if pd.notna(v) else None))
        elif dtype == "bool":
            converters.append(lambda v: (bool(v) if pd.notna(v) else None))
        elif dtype.startswith(("int", "float")):
            converters.append(lambda v: (v if pd.notna(v) else None))
        else:
            converters.append(lambda v: (str(v) if pd.notna(v) else None))

    def to_row(t):
        return tuple(conv(val) for conv, val in zip(converters, t))

    tuples_iter = (to_row(t) for t in df.itertuples(index=False, name=None))
    from itertools import islice
    peek = list(islice(tuples_iter, 1))
    if peek:
        print(f"🔍 [4/5] STEP 4e: First row preview: {peek[0]}")

    def chain_peek_and_rest():
        if peek:
            yield peek[0]
        for t in tuples_iter:
            yield t

    def chunks(iterable, size):
        it = iter(iterable)
        for first in it:
            yield [first] + list(islice(it, size - 1))

    total = 0
    for batch in chunks(chain_peek_and_rest(), batch_size):
        cur.executemany(insert_sql, batch)
        total += len(batch)
        print(f"✅ [4/5] STEP 4f: Inserted {len(batch)} rows (total {total}) into {table}")

    print(f"🎉 [4/5] STEP 4g: Finished inserting {total} rows into {table}")
    cur.close()


In [7]:
def drop_table(table: str = REPORT_TABLE):
    print(f"🗑️ [X] Dropping table if exists: {table} ...")
    cursor = conn.cursor()
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
        print(f"✅ [X] Table {table} dropped successfully.")
    except Exception as e:
        print(f"❌ [X] Failed to drop table {table}: {e}")
    finally:
        cursor.close()

In [8]:
# # --------------------
# # Step 4: Run for a single day
# # --------------------
# date_str = "2025-09-10"
# print(f"▶️ [0/5] Starting job for {date_str}")

# df = fetch_data_for_day(conn, date_str)
# # df = process_soc_charging_data(df)
# # write_df_to_iceberg(df)

# # --------------------
# # Step 5: Close connection
# # --------------------
# print("🔒 [5/5] STEP 5: Closing Trino connection...")
# conn.close()
# print("✅ [5/5] STEP 5: Connection closed. Job complete.")

In [9]:
#################################################################################
### Execute the following lines for the first time to gather historical data ###
#################################################################################
# drop_table()

START_DATE = date(2025, 9, 1)
YESTERDAY = date.today() - timedelta(days=1)

d = START_DATE
idx = 1

# Initialize an empty list to store DataFrames
df_list = []

while d <= YESTERDAY:
    date_str = d.isoformat()
    print(f"▶️ [{idx}] Starting job for {date_str}")

    df = fetch_data_for_day(conn, date_str)
    # df = process_soc_charging_data(df)

    d += timedelta(days=1)
    idx += 1
    
    if df.empty:
        print(f"No charging events were detected for device {device_id}.")
        continue        
    else:
        # write_df_to_iceberg(df)
        # Append the DataFrame to the list
        df_list.append(df)        


# Concatenate all DataFrames in the list at once
df_combined = pd.concat(df_list, ignore_index=True)

# --------------------
# Step 5: Close connection
# --------------------
print("🔒 [5/5] STEP 5: Closing Trino connection...")
conn.close()
print("✅ [5/5] STEP 5: Connection closed. Job complete.")

▶️ [1] Starting job for 2025-09-01
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09-01...
⚙️ [2/5] STEP 2b: Executing query for 2025-09-01 and 2025-08-31...
📊 [2/5] STEP 2c: Rows fetched for the two-day period: 189593
▶️ [2] Starting job for 2025-09-02
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09-02...
⚙️ [2/5] STEP 2b: Executing query for 2025-09-02 and 2025-09-01...
📊 [2/5] STEP 2c: Rows fetched for the two-day period: 136412
▶️ [3] Starting job for 2025-09-03
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09-03...
⚙️ [2/5] STEP 2b: Executing query for 2025-09-03 and 2025-09-02...
📊 [2/5] STEP 2c: Rows fetched for the two-day period: 112161
▶️ [4] Starting job for 2025-09-04
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09-04...
⚙️ [2/5] STEP 2b: Executing query for 2025-09-04 and 2025-09-03...
📊 [2/5] STEP 2c: Rows fetched for the two-day period: 125957
▶️ [5] Starting job for 2025-09-05
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09

In [14]:
df_combined.loc[:, 'date'] = df_combined['IST'].dt.date
# df_combined.groupby(['date','id']).size().reset_index(name='count_of_instances')
df_combined[df_combined['id'].isin(['3','16'])].groupby(['id', 'date']).size().reset_index(name='count_of_instances')

,id,date,count_of_instances
0,16,2025-09-01,71377
1,16,2025-09-02,49757
2,16,2025-09-03,43590
3,16,2025-09-04,50621
4,16,2025-09-05,47811
5,16,2025-09-06,44236
6,16,2025-09-07,52465
7,16,2025-09-08,44124
8,16,2025-09-09,49708
9,16,2025-09-10,45958


In [11]:
wqeqw

NameError: name 'wqeqw' is not defined

In [ ]:
df_combined.loc[:, 'date'] = df_combined['start_time'].dt.date
df_combined[df_combined['vehicle_id'].isin(['3','16','18','19'])].groupby(['date','vehicle_id']).size().reset_index(name='count_of_instances')

In [ ]:
wait here

In [ ]:
hasda

In [ ]:
df_combined[df_combined['vehicle_id'].isin(['3','16','18','19'])]

In [ ]:
# Extract the date part from the 'start_time' column
df_combined.loc[:, 'start_date'] = df_combined['start_time'].dt.date

In [ ]:
df_combined[df_combined['vehicle_id'].isin(['3','16','18','19'])].groupby(['vehicle_id', 'start_date']).size().reset_index(name='count_of_instances')

In [ ]:
df_combined.IST.max()

In [ ]:
# #################################################################################
# ### Execute the following lines to execute the daily fetch and process tasks ###
# #################################################################################
# yesterday = date.today() - timedelta(days=1)
# date_str = yesterday.isoformat()
# df = fetch_data_for_day(conn, date_str)
# df = process_soc_charging_data(df)
# # write_df_to_iceberg(df)


# # --------------------
# # Step 5: Close connection
# # --------------------
# print("🔒 [5/5] STEP 5: Closing Trino connection...")
# conn.close()
# print("✅ [5/5] STEP 5: Connection closed. Job complete.")

In [ ]:
df.head()

In [ ]:
df1.IST.min()

In [ ]:
df1.IST.max()

In [18]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import trino
from datetime import datetime, date, timedelta
from db_operations import connect_to_trino, write_df_to_iceberg

In [77]:
# ---- reporting config (edit ONLY this) ----
TABLE_NAME = "power_consumption_report"   # <— This is now the source of truth for the table name


# --------------------
# Step 2: Function to fetch data for a given day
# --------------------
def fetch_data_for_day(conn, date_str: str, ids: list = None) -> pd.DataFrame:

    print(f"📥 [2/5] STEP 2a: Validating and fetching data for {date_str}...")

    target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
    yesterday_date = target_date - timedelta(days=1)

    # Conditional WHERE clause based on the 'ids' parameter
    where_clause = ""
    if ids is not None and len(ids) > 0:
        id_list_str = ", ".join(f"'{id}'" for id in ids)
        where_clause = f"AND id IN ({id_list_str})"

    query = f"""
        WITH two_days_data AS (
            SELECT 
                "id",
                at_timezone("timestamp", 'Asia/Kolkata') AS IST,
                "BAT_SOC", "Bat_Voltage", "Total_Battery_Current", "GUN_Connection_Status",
                "OdoMeterReading", "Gear_Position", "Vehiclereadycondition",
                "Chargingcontactor1positive", "Chargingcontactor1negative",
                "Chargingcontactor2positive", "Chargingcontactor2negative"
            FROM
                facts_prod.can_parsed_output_100
            WHERE
                dt = DATE '{target_date.isoformat()}' OR dt = DATE '{yesterday_date.isoformat()}'
                {where_clause}                
        )
        SELECT
            *
        FROM
            two_days_data
        WHERE
            CAST(IST AS DATE) = DATE '{target_date.isoformat()}'
    """
    
    cursor = conn.cursor()
    print(f"⚙️ [2/5] STEP 2b: Executing query for {target_date} and {yesterday_date}...")
    cursor.execute(query)

    print("✅ [2/5] STEP 2c: Query executed successfully!")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=columns)

    if not df.empty:
        # df['id'] = df['id'].astype(str)
        # df['Total_Battery_Current'] = pd.to_numeric(df['Total_Battery_Current'], errors='coerce')
        # df.dropna(subset=['Total_Battery_Current'], inplace=True)
        df['Total_Battery_Current'] = df['Total_Battery_Current'].astype(float)
        print(f"✅ [2/5] STEP 2d: Data fetching for {date_str} completed. Rows fetched: {len(df)}")
    else:
        print(f"⚠️ [2/5] STEP 2d: No data found for {date_str}. Returning empty DataFrame.")
    

    return df

In [95]:
def process_soc_charging_data(df: pd.DataFrame):
    """
    Generates a summary, and performs outlier analysis for charging events
    across multiple device IDs.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing raw data for one or more devices.
        
    Returns:
        pd.DataFrame: A DataFrame summarizing charging events for each device.
    """

    # Create a copy of the DataFrame to avoid modifying a slice.
    df = df.copy()    
    
    if df.empty or 'id' not in df.columns:
        print("Input DataFrame is empty or does not contain an 'id' column.")
        return pd.DataFrame()
        
    device_ids = df['id'].unique().tolist()
    all_summary_data = []

    for device_id in device_ids:
        device_df = df[df['id'] == device_id].copy()

        # Check if the grouped DataFrame is empty
        if device_df.empty:
            print(f"No charging events were detected for device {device_id}.")
            continue        
        
        if 'timestamp' in device_df.columns:
            device_df.loc[:, 'ist_timestamp'] = pd.to_datetime(device_df['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        elif 'IST' in device_df.columns:
            device_df.loc[:, 'ist_timestamp'] = pd.to_datetime(device_df['IST'])
        else:
            print(f"Neither 'timestamp' nor 'IST' column found for device {device_id}. Cannot proceed.")
            continue
            
        device_df.sort_values(by='ist_timestamp', inplace=True)
        device_df.reset_index(drop=True, inplace=True)

        current_threshold = 3200
        device_df = device_df[(device_df['Total_Battery_Current'].abs() != current_threshold)].copy()
        
        for col in ['BAT_SOC', 'Bat_Voltage', 'Chargingcontactor1positive',
                    'Chargingcontactor1negative', 'Chargingcontactor2positive',
                    'Chargingcontactor2negative', 'GUN_Connection_Status']:
            device_df.loc[:, col] = device_df[col].fillna(0)
        
        for col in ['BAT_SOC', 'Bat_Voltage']:
            device_df.loc[:, col] = device_df[col].replace(0.0, np.nan).ffill().bfill()
        
        device_df.loc[:, 'Total_Battery_Current'] = device_df['Total_Battery_Current'].fillna(0)

        # --- MODIFICATION START ---
        # Use GUN_Connection_Status for session detection
        charging_start_indices = device_df[device_df['GUN_Connection_Status'].diff() == 1].index.tolist()
        charging_end_indices = device_df[device_df['GUN_Connection_Status'].diff() == -1].index.tolist()
        # --- MODIFICATION END ---
        
        if not charging_start_indices and not device_df.empty and device_df.iloc[0]['GUN_Connection_Status'] == 1:
            charging_start_indices.insert(0, device_df.index[0])
        
        if len(charging_end_indices) < len(charging_start_indices):
            if len(charging_start_indices) > 0:
                charging_end_indices.append(device_df.index[-1])
            
        merged_events = []
        if len(charging_start_indices) > 0 and len(charging_end_indices) > 0:
            num_sessions = min(len(charging_start_indices), len(charging_end_indices))

            for i in range(num_sessions):
                start_index = charging_start_indices[i]
                end_index = charging_end_indices[i]
                
                if i > 0:
                    prev_end_time = device_df.loc[charging_end_indices[i-1], 'ist_timestamp']
                    current_start_time = device_df.loc[start_index, 'ist_timestamp']
                    prev_end_soc = device_df.loc[charging_end_indices[i-1], 'BAT_SOC']
                    current_start_soc = device_df.loc[start_index, 'BAT_SOC']

                    time_diff = (current_start_time - prev_end_time).total_seconds()
                    soc_diff = abs(current_start_soc - prev_end_soc)
                    
                    if (time_diff <= 5 * 60 and soc_diff <= 1.0) or (time_diff <= 60):
                        merged_events[-1] = (merged_events[-1][0], end_index)
                        continue
                
                merged_events.append((start_index, end_index))
        else:
            print(f"No charging events were detected for device {device_id}.")
            continue

        summary_data_device = []
        BATTERY_CAPACITY_KWH = 423
        
        for start_index, end_index in merged_events:
            event_df = device_df.loc[start_index:end_index].copy()

            if event_df.empty:
                print(f"Warning: Empty event data found for device {device_id}. Skipping.")
                continue            
                
            # MODIFICATION: Use 'GUN_Connection_Status' to filter for charging periods
            charging_periods = event_df[event_df['GUN_Connection_Status'] == 1].copy()
            
            total_duration = 0
            if not charging_periods.empty:
                charging_periods.loc[:, 'time_diff'] = charging_periods['ist_timestamp'].diff().dt.total_seconds().fillna(0)
                total_duration = int(charging_periods['time_diff'].sum())

            start_row = event_df.iloc[0].copy()
            end_row = event_df.iloc[-1].copy()
            
            energy_Wh = 0
            if not charging_periods.empty:
                charging_periods.loc[:, 'power_W'] = charging_periods['Bat_Voltage'] * charging_periods['Total_Battery_Current'].abs()
                energy_Wh = np.trapezoid(charging_periods['power_W'], x=charging_periods['ist_timestamp'].astype(np.int64) / 10**9) / 3600
            
            total_kwh_consumed_tpc = energy_Wh / 1000

            total_kwh_consumed_soc = (end_row['BAT_SOC'] - start_row['BAT_SOC']) * BATTERY_CAPACITY_KWH / 100
            total_kwh_consumed_soc = abs(total_kwh_consumed_soc)

            percent_diff = 0
            if total_kwh_consumed_tpc + total_kwh_consumed_soc != 0:
                percent_diff = (abs(total_kwh_consumed_tpc - total_kwh_consumed_soc) / 
                                ((total_kwh_consumed_tpc + total_kwh_consumed_soc) / 2)) * 100
            
            summary_data_device.append({
                'vehicle_id': device_id,
                'start_time': start_row['ist_timestamp'],
                'end_time': end_row['ist_timestamp'],
                'charge_dur_seconds': round(total_duration,0),
                'soc_start': start_row['BAT_SOC'],
                'soc_end': end_row['BAT_SOC'],
                'tpc_kwh': round(total_kwh_consumed_tpc,2),
                'soc_kwh': round(total_kwh_consumed_soc,2),
                'diff_kwh_percent': round(percent_diff,2)
            })
        
        all_summary_data.extend(summary_data_device)
            
    return pd.DataFrame(all_summary_data)

In [96]:
# --------------------
# Main execution logic
# --------------------

conn = connect_to_trino()

if conn:
    try:
        # Define the vehicle IDs for the report
        vehicle_ids_for_report = ['3', '16', '18', '19']

        # Get the date to process
        yesterday = date.today() - timedelta(days=1)
        date_str = yesterday.isoformat()
        
        print(f"▶️ Starting daily report job for {date_str}")

        # Fetch data using the logic from this file
        df_raw = fetch_data_for_day(conn, date_str, vehicle_ids_for_report)
        
        if not df_raw.empty:
            # Process the data
            df_processed = process_soc_charging_data(df_raw)
            
            if not df_processed.empty:
                # Write to the database (uncomment to enable)
                # write_df_to_iceberg(df_processed)
                print("Processing complete. DataFrame ready to be written to Iceberg.")
            else:
                print("Processed DataFrame is empty. No data to write.")
        else:
            print("Raw DataFrame is empty. No processing needed.")

    finally:
        print("🔒 [5/5] STEP 5: Closing Trino connection...")
        conn.close()
        print("✅ [5/5] STEP 5: Connection closed.")
else:
    print("Failed to establish a database connection. Exiting.")

🔌 [1/5] STEP 1: Connecting to Trino...
✅ [1/5] STEP 1: Connected to Trino
▶️ Starting daily report job for 2025-09-14
📥 [2/5] STEP 2a: Validating and fetching data for 2025-09-14...
⚙️ [2/5] STEP 2b: Executing query for 2025-09-14 and 2025-09-13...
✅ [2/5] STEP 2c: Query executed successfully!
✅ [2/5] STEP 2d: Data fetching for 2025-09-14 completed. Rows fetched: 302586
No charging events were detected for device 6.
No charging events were detected for device 27.
Processing complete. DataFrame ready to be written to Iceberg.
🔒 [5/5] STEP 5: Closing Trino connection...
✅ [5/5] STEP 5: Connection closed.


In [97]:
df_processed.head()

,vehicle_id,start_time,end_time,charge_dur_seconds,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kw_percent
0,11,2025-09-14 06:54:03.231000+05:30,2025-09-14 08:53:48.386000+05:30,5619,39.2,100.0,251.72,257.18,2.15
1,11,2025-09-14 15:13:24.750000+05:30,2025-09-14 15:36:08.750000+05:30,1360,57.6,74.4,72.63,71.06,2.18
2,11,2025-09-14 18:59:36.242000+05:30,2025-09-14 20:12:44.942000+05:30,4386,40.4,100.0,261.45,252.11,3.64
3,13,2025-09-14 06:00:05.132000+05:30,2025-09-14 07:26:18.112000+05:30,4020,44.4,100.0,243.63,235.19,3.53
4,13,2025-09-14 14:08:17.547000+05:30,2025-09-14 14:44:19.054000+05:30,2159,52.0,80.0,124.33,118.44,4.86


In [98]:
df_processed[df_processed.diff_kw_percent>10]

,vehicle_id,start_time,end_time,charge_dur_seconds,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kw_percent
10,15,2025-09-14 10:07:00.417000+05:30,2025-09-14 10:31:59.797000+05:30,1496,91.2,100.0,30.38,37.22,20.23
15,16,2025-09-14 18:20:57.532000+05:30,2025-09-14 20:23:30.961000+05:30,7352,17.6,100.0,300.23,348.55,14.89
17,3,2025-09-14 07:29:58.021000+05:30,2025-09-14 07:34:34.161000+05:30,275,97.6,100.0,5.94,10.15,52.33
20,7,2025-09-14 07:56:11.409000+05:30,2025-09-14 08:40:14.038000+05:30,808,96.0,100.0,6.02,16.92,95.04
24,9,2025-09-14 15:04:37.959000+05:30,2025-09-14 15:35:32.440000+05:30,1852,56.0,80.0,114.35,101.52,11.88
27,19,2025-09-14 06:11:44.553000+05:30,2025-09-14 06:16:18.713000+05:30,274,61.6,63.6,9.68,8.46,13.41
29,19,2025-09-14 06:30:42.454000+05:30,2025-09-14 06:30:44.973000+05:30,2,70.4,70.4,0.09,0.00,200.00
30,19,2025-09-14 07:36:57.373000+05:30,2025-09-14 07:37:00.033000+05:30,2,72.8,72.8,0.09,0.00,200.00
31,19,2025-09-14 20:43:02.281000+05:30,2025-09-14 20:44:43.901000+05:30,101,74.0,74.0,0.18,0.00,200.00


In [88]:
df_processed.loc[:, 'date'] = df_processed['start_time'].dt.date
# df_combined.groupby(['date','id']).size().reset_index(name='count_of_instances')
df_processed[df_processed['vehicle_id'].isin(['3','16'])].groupby(['vehicle_id', 'date']).size().reset_index(name='count_of_instances')

,vehicle_id,date,count_of_instances
0,16,2025-09-14,2
1,3,2025-09-14,3


In [89]:
df_raw.loc[:, 'date'] = df_raw['IST'].dt.date
# df_combined.groupby(['date','id']).size().reset_index(name='count_of_instances')
df_raw[df_raw['id'].isin(['3','16'])].groupby(['id', 'date']).size().reset_index(name='count_of_instances')

,id,date,count_of_instances
0,16,2025-09-14,49740
1,3,2025-09-14,48184
